# Introduction to TensorFlow

# Table of Contents

1. Graph and session
2. Tensor <br>
    2.1. Constants, Sequences, and Random Values <br>
    2.2. Variables <br>
    2.3. Placeholders
5. Saving and Restoring

# 1. Graph and session

In [3]:
import tensorflow as tf

In [10]:
a = tf.constant(5)
b = tf.constant(3)
c = tf.multiply(a, b)

sess = tf.Session()

print(sess.run(c))

sess.close()

with tf.Session() as sess:
    
    print(sess.run(c))

15
15


# 2. Tensors

## 2.1. Constants, Sequences, and Random Values

In [ ]:
# tensorboard --logdir=PATH

In [11]:
a = tf.constant(1, name="a")
b = tf.constant(3, name="b")
x = tf.add(a, b, name="add")

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(x))
writer.close()

4


In [16]:
tf.reset_default_graph()

a = tf.constant([[2, 2]], name="a")
b = tf.constant([[0, 1], [2, 3]], name="b")

x = tf.add(a, b, name="add")
y = tf.multiply(a, b, name="mul")

z = tf.matmul(a, b, name="matmul")

with tf.Session() as sess:
    
    print(sess.run(x))
    print()
    print(sess.run(y))
    print()
    print(sess.run(z))

[[2 3]
 [4 5]]

[[0 2]
 [4 6]]

[[4 8]]


In [19]:
a = tf.zeros(shape=[2, 1], dtype=tf.int32, name="a")
b = tf.fill([2, 1, 3], 4.0)

with tf.Session() as sess:
    
    print(sess.run(a))
    print()
    print(sess.run(b))

[[0]
 [0]]

[[[4. 4. 4.]]

 [[4. 4. 4.]]]


In [22]:
a = tf.linspace(10.0, 12.0, 5)
b = tf.range(1, 10, 2)

with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))

[10.  10.5 11.  11.5 12. ]
[1 3 5 7 9]


In [23]:
norm = tf.random_normal([2, 3], mean=-1, stddev=4)
unif = tf.random_uniform([1, 3], minval=0, maxval=10)

with tf.Session() as sess:
    print(sess.run(norm))
    print()
    print(sess.run(unif))

[[-10.481074   -1.9077125  -4.1953936]
 [ -3.3815193   2.1222029   2.3484426]]

[[5.637683  4.143921  3.3832204]]


## 2.2. Variables

In [36]:
a = tf.Variable(2, name="scalar")
b = tf.Variable([[0, 1], [2, 3]], name="matrix")
c = tf.Variable(tf.zeros([2, 2], dtype=tf.int32), name="zeros")
d = tf.Variable(tf.random_uniform([1, 3], minval=0, maxval=10), name="random_init")
# init_ab = tf.variable_initializer([a, b], name="init_ab")
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    # sess.run(init_ab)
    sess.run(init_op)
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))
    print(sess.run(d))

2
[[0 1]
 [2 3]]
[[0 0]
 [0 0]]
[[8.33042  8.724636 4.703435]]


## 2.3. Placeholders

In [43]:
tf.reset_default_graph()

x = tf.placeholder(shape=[1, 3], dtype=tf.float32, name="x")
y = tf.matmul(x, tf.transpose(x), name="y")
z = tf.multiply(x, 2, name="z")

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(y, feed_dict={x: [[1, 2, 3]]}))
    print(sess.run(z, feed_dict={x: [[1, 2, 3]]}))
    # print(sess.run([y, z] feed_dict={x: [[1, 2, 3]]}))
    for i in range(10):
        print(sess.run(z, feed_dict={x: [[i, i, i]]}))

writer.close()

[[14.]]
[[2. 4. 6.]]
[[0. 0. 0.]]
[[2. 2. 2.]]
[[4. 4. 4.]]
[[6. 6. 6.]]
[[8. 8. 8.]]
[[10. 10. 10.]]
[[12. 12. 12.]]
[[14. 14. 14.]]
[[16. 16. 16.]]
[[18. 18. 18.]]


# 3. Saving and Restoring

In [52]:
tf.reset_default_graph()

a = tf.Variable(tf.random_normal([1]), name="a")
b = tf.Variable(tf.random_normal([1]), name="b")

x = tf.placeholder(shape=[1], dtype=tf.float32, name="x")

y = tf.add(tf.multiply(a, x), b, name="y")

init_op = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init_op)
    print(sess.run([a, b]))
    print(sess.run(y, feed_dict={x: [2.0]}))
    
    saver.save(sess, './models/modellll')

[array([-1.6198103], dtype=float32), array([-1.0922256], dtype=float32)]
[-4.331846]


In [53]:
with tf.Session() as sess:
    model = tf.train.import_meta_graph('./models/modellll.meta')
    model.restore(sess, tf.train.latest_checkpoint('./models/'))
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    y = graph.get_tensor_by_name("y:0")
    print(sess.run(y, feed_dict={x: [2.0]}))

INFO:tensorflow:Restoring parameters from ./models/modellll
[-4.331846]
